In [1]:
%%bash
#To start, you'll need a file containing the geometry of ground state (aka, at the Franck-Condon point)
cat base_geometries/original_cartesian_coordinates/fc.xyz


10

C        -1.71206       -0.32706        0.11160
C        -0.30557       -0.48176        0.35394
C         0.75752       -0.01615       -0.49967
C         2.05157       -0.10807       -0.17313
H        -2.06407        0.20018       -0.76653
H        -2.42291       -0.66969        0.85372
H        -0.03213       -0.98685        1.27816
H         0.47549        0.43131       -1.44989
H         2.38131       -0.53852        0.76687
H         2.81691        0.26094       -0.84850


In [2]:
#Here's a gif of the molecule's FC geometry
from IPython.display import HTML
HTML('<img src="base_geometries/original_cartesian_coordinates/fc.gif">')

In [1]:
%%bash
#We'll also need a target geometry of the same molecule. For this demo, we'll use the transoid conical intersection.
cat base_geometries/original_cartesian_coordinates/Transoid.xyz

10

C    0.0099793130   -0.0426199469    0.0334010820
C    1.4861157025   -0.0976854547    0.0762123373
C    2.2613530886    1.0972557235   -0.1077587602
C    2.8086804038    0.9763007590    1.2397385526
H   -0.4698842455    0.3640063373   -0.8342116499
H   -0.5810457210   -0.2205813910    0.9080009880
H    1.9848176513   -1.0325999267    0.2159134468
H    1.8101598372    2.0061670885   -0.4459653405
H    2.4257596824    1.5802733808    2.0391556726
H    3.5192864494    0.2117836737    1.4794448263


In [4]:
#Here's a gif of the molecule's transoid geometry
from IPython.display import HTML
HTML('<img src="base_geometries/original_cartesian_coordinates/Transoid.gif">')